In [63]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
from sortedcontainers import SortedDict
import matplotlib.pyplot as plt
import seaborn as sns
from  matplotlib.ticker import PercentFormatter
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics

In [64]:
# settings
name = "Dblp"
come_model_type = "BGMM"
ks = [2, 5]  # [2, 5, 10, 20]

In [65]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)

In [66]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_k-{k}_ds-0.0_type-{come_model_type}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels = pd.read_csv(f"./data/labels_pred_{come_model_type}_{k}.txt", header=None)
    labels = labels.rename(columns={0: 'label'})
    labels.label = labels.astype(int)

    # join labels to embeddings
    df = df.join(labels)

    display(df)
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,1,2,3,4,5,6,7,8,9,10,...,120,121,122,123,124,125,126,127,128,label
node,,,,,,,,,,,,,,,,,,,,,
0,0.447244,-0.590060,-0.327605,0.759601,0.407560,0.058691,0.100031,0.519430,0.105071,0.309200,...,0.643391,0.026815,-0.170982,-0.574228,-0.198281,-0.451050,0.011072,-0.094810,-0.301875,1
1,0.295918,-0.558974,-0.200630,0.670498,0.255454,0.012444,0.046877,0.438116,-0.062488,0.382698,...,0.522785,-0.077961,-0.241738,-0.520454,-0.098120,-0.499365,-0.048436,0.035603,-0.250949,1
2,0.533605,-0.535543,-0.390154,0.778972,0.445067,0.030342,0.050596,0.546266,0.181827,0.356746,...,0.632027,0.023890,-0.090602,-0.596374,-0.249288,-0.443732,-0.064824,-0.046458,-0.358123,1
3,0.422736,-0.478823,-0.391336,0.637420,0.339128,-0.047656,0.112250,0.464919,0.160469,0.317239,...,0.552122,-0.017747,-0.144513,-0.465714,-0.285970,-0.305572,0.000968,-0.058966,-0.357066,1
4,0.325676,-0.428440,-0.432361,0.703664,0.361787,0.000723,-0.070736,0.472260,0.272814,0.312631,...,0.598659,0.039992,-0.145147,-0.551694,-0.296413,-0.290567,-0.063562,-0.169452,-0.348867,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.257980,-0.395774,-0.185478,0.552952,0.410871,-0.034553,-0.067511,0.413107,0.183474,0.325979,...,0.546140,-0.171239,-0.060339,-0.347124,-0.287109,-0.128301,0.030241,0.094815,-0.255052,1
13180,-0.020099,-0.540597,-0.269626,0.003425,0.431760,0.091559,0.119777,0.100602,0.358634,0.445394,...,0.096251,-0.318657,0.178803,-0.034317,0.113252,0.093031,-0.522664,0.109797,0.340559,0
13181,-0.118595,-0.484969,-0.095687,0.009911,0.409799,-0.006928,0.062702,0.008954,0.505634,0.593392,...,0.109613,-0.070694,0.259097,-0.084647,0.026951,-0.060800,-0.500006,0.138947,0.363550,0


,1,2,3,4,5,6,7,8,9,10,...,120,121,122,123,124,125,126,127,128,label
node,,,,,,,,,,,,,,,,,,,,,
0,0.434226,-0.387976,-0.342012,0.727168,0.339455,0.211082,-0.027637,0.511111,0.133573,0.330798,...,0.622809,0.135571,-0.293498,-0.539234,-0.265420,-0.379549,-0.110447,-0.076284,-0.343553,2
1,0.265924,-0.362547,-0.275657,0.634993,0.248291,0.214578,-0.060368,0.525814,-0.084931,0.387744,...,0.515422,0.069670,-0.297604,-0.484533,-0.044852,-0.506817,-0.237709,0.076055,-0.296019,2
2,0.539574,-0.342830,-0.470852,0.709976,0.387971,0.104439,-0.028320,0.518842,0.274218,0.313852,...,0.635879,0.095333,-0.179778,-0.516522,-0.338626,-0.330501,-0.224829,-0.066941,-0.375785,2
3,0.404247,-0.354206,-0.359028,0.522853,0.318014,0.021533,0.011222,0.436997,0.208020,0.299709,...,0.512195,0.097435,-0.197099,-0.393147,-0.290320,-0.291386,-0.063158,-0.024950,-0.379730,2
4,0.292309,-0.274753,-0.430262,0.620953,0.283817,0.120868,-0.159357,0.449058,0.287632,0.281005,...,0.567311,0.191719,-0.216171,-0.459852,-0.342050,-0.281305,-0.127021,-0.099702,-0.363577,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.309951,-0.495111,-0.300779,0.458932,0.400343,-0.025314,-0.124034,0.430032,0.282203,0.353015,...,0.635563,-0.128318,-0.118762,-0.254270,-0.394024,-0.180928,-0.113551,0.027718,-0.182381,2
13180,-0.179346,-0.425092,-0.131352,-0.008551,0.641813,0.157232,-0.034580,0.084456,0.400029,0.587169,...,0.065176,-0.193749,0.104673,-0.087115,0.047355,0.194254,-0.676473,0.227743,0.301922,1
13181,-0.202129,-0.270683,-0.065851,-0.086173,0.563360,0.147709,-0.030454,0.010829,0.400087,0.621894,...,0.115228,-0.073927,0.141782,-0.137634,0.127659,0.144769,-0.535276,0.204934,0.395553,1


# Community Detection

In [67]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

K = 2
NMI:  0.44116900858585945
K = 5
NMI:  0.6498711817255415


In [68]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label == i].index.to_numpy()
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

K = 2
  i0 conductance: 0.026228756671796718
  i1 conductance: 0.026228756671796718
=>conductance: 0.026228756671796718
K = 5
  i0 conductance: 0.24569508609827803
  i1 conductance: 0.056777346557424176
  i2 conductance: 0.08955412403688266
  i3 conductance: 0.010057981303987693
  i4 conductance: 0.06873344622465373
=>conductance: 0.010057981303987693


# Node Classification

In [ ]:
# train LibSVM classifier on 90% of data and test on 10%.

#TODO

In [ ]:
def emb_from_row(row):
    """get node embedding np array from df row"""
    return row.drop('label').to_numpy().flatten()
